In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.50
#Caso con edad en mean en vez de 200 para los nan unicamente
#nuevos features

%matplotlib inline

In [2]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [3]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [4]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [5]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [6]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [7]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,nombre_area,tipo_de_trabajo,nivel_laboral
0,18,BolNL,0.050000,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN


In [8]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones.sample(len(postulaciones)*3//4), ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

C:\Users\Pocho\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


67

In [9]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

7

In [10]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [11]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
2012214,1112441709,ZDaoAdR,1,41.0,True,False,False,False,False,False,...,False,Full-time,Junior,24.0,0.0,0.000000,10.0,18.0,18.0,False
3997725,1111137355,wVXlAAl,0,29.0,True,False,False,False,False,False,...,True,Part-time,Senior / Semi-Senior,27.0,0.0,0.000000,0.0,6.0,6.0,False
1025135,1112363711,96zp8xY,1,25.0,True,False,False,False,False,False,...,False,Part-time,Otro,44.0,1.0,0.022727,1.0,7.0,7.0,True
412192,1112425382,QNMeGvW,1,22.0,True,False,False,False,False,False,...,False,Full-time,-,216.0,0.0,0.000000,1.0,181.0,181.0,False
659857,1112349544,ez2GQj,1,32.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False
2188723,1112147486,ekmKWX9,1,52.0,False,True,False,False,False,False,...,False,Full-time,Jefe / Supervisor / Responsable,3.0,0.0,0.000000,1.0,3.0,3.0,False
1269778,1112345158,evMBM2,1,37.0,False,True,False,False,False,False,...,False,Full-time,Junior,38.0,0.0,0.000000,7.0,32.0,32.0,False
3518475,1112512608,aG2pAN,0,29.0,False,True,False,False,False,False,...,False,Part-time,Junior,4.0,0.0,0.000000,0.0,0.0,0.0,False
157677,1112094756,bOjBoK2,1,27.0,False,True,False,False,False,False,...,True,Part-time,Senior / Semi-Senior,10.0,1.0,0.100000,1.0,1.0,1.0,True
2549322,1112466548,akjm659,0,28.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False


In [12]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [13]:
postulaciones.dropna(axis = 1, inplace=True)

In [14]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Facturación,1
1,0z5Dmrd,Logística,1
2,0z5Dmrd,Medicina,1
3,0z5Dmrd,Otros,1
4,0z5Dmrd,Tecnologia / Sistemas,1


In [15]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [16]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
1168227,1112150291,EzppkW6,0,23.0,True,False,False,False,False,False,...,13.0,0.0,0.000000,0.0,10.0,10.0,False,1,8,1
673622,1112342656,dYz5LMq,1,20.0,False,True,False,False,False,False,...,134.0,0.0,0.000000,9.0,25.0,25.0,False,10,31,38
2027233,1112303993,rmdWmk1,1,32.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,9,19,12
5217105,1112364691,ZxKQk5,0,38.0,True,False,False,False,False,False,...,3.0,0.0,0.000000,0.0,3.0,3.0,False,0,10,3
1561684,1112306651,A3AdZ55,1,23.0,True,False,False,False,False,False,...,6.0,0.0,0.000000,3.0,6.0,6.0,False,1,8,5
1539472,1112430722,RzrBbXo,1,26.0,False,True,False,False,False,False,...,78.0,1.0,0.012821,8.0,65.0,65.0,True,4,34,8
2372944,1112267946,5mkDrep,1,25.0,False,True,False,False,False,False,...,7.0,0.0,0.000000,1.0,7.0,7.0,False,14,60,14
5153075,1112487850,2zQP83Y,0,28.0,False,True,False,False,False,False,...,8.0,0.0,0.000000,0.0,7.0,7.0,False,1,20,18
2642507,1112286220,qeYkppb,1,24.0,False,True,False,False,False,False,...,89.0,0.0,0.000000,0.0,73.0,73.0,False,0,69,15
981481,1112335409,EzlNo90,1,24.0,True,False,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,3,9,1


In [18]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')
#features.remove("ciudad")
#features.remove("mapacalle")

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5011767 Test:  556864


7

In [19]:
model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[0]	validation_0-error:0.13385
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.106557
[2]	validation_0-error:0.103602
[3]	validation_0-error:0.100455
[4]	validation_0-error:0.098651
[5]	validation_0-error:0.097467
[6]	validation_0-error:0.09646
[7]	validation_0-error:0.096433
[8]	validation_0-error:0.095955
[9]	validation_0-error:0.094772
[10]	validation_0-error:0.094307
[11]	validation_0-error:0.093901
[12]	validation_0-error:0.09347
[13]	validation_0-error:0.09324
[14]	validation_0-error:0.092637
[15]	validation_0-error:0.092186
[16]	validation_0-error:0.091301
[17]	validation_0-error:0.091062
[18]	validation_0-error:0.090995
[19]	validation_0-error:0.090774
[20]	validation_0-error:0.090417
[21]	validation_0-error:0.090135
[22]	validation_0-error:0.089688
[23]	validation_0-error:0.089402
[24]	validation_0-error:0.089226
[25]	validation_0-error:0.088927
[26]	validation_0-error:0.088639
[27]	validation_0-error:0.088427
[28]	validation_0-error:0

In [20]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9825994321836056


In [21]:
print("dale mono")

dale mono


In [22]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean5.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean5.pkl']

In [23]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.sample(10)

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
89358,70365,1112461742,kPLkE2V,29.0,False,True,False,False,False,False,...,86.0,1.0,0.011628,True,1.0,13.0,7.0,6.0,73.0,73.0
84206,32938,1112398585,aLoGYq,28.0,False,True,False,False,False,False,...,13.0,0.0,0.000000,False,3.0,0.0,8.0,8.0,1.0,1.0
41015,29560,1112372247,JBxR1W3,22.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,31.0,48.0,30.0,0.0,0.0,0.0
78285,38845,1112416054,8J4eaO,53.0,True,False,False,False,False,False,...,200.0,0.0,0.000000,False,13.0,50.0,2.0,29.0,163.0,163.0
66432,40232,1112418720,3NdYqxa,64.0,False,True,False,False,False,False,...,8.0,1.0,0.125000,True,0.0,5.0,3.0,3.0,3.0,3.0
74541,20627,1112260086,aZLZOZ,38.0,True,False,False,False,False,False,...,2.0,0.0,0.000000,False,0.0,5.0,6.0,0.0,2.0,2.0
37337,59173,1112453552,MVPR5AK,35.0,True,False,False,False,False,False,...,1.0,0.0,0.000000,False,4.0,11.0,7.0,1.0,1.0,1.0
68804,43138,1112424633,ZOaJxP,47.0,False,True,False,False,False,False,...,13.0,0.0,0.000000,False,0.0,7.0,7.0,0.0,10.0,10.0
329,66848,1112460141,xkvOXG6,36.0,False,True,False,False,False,False,...,35.0,0.0,0.000000,False,11.0,32.0,21.0,13.0,35.0,35.0
33711,45611,1112428517,2zmlLx1,31.0,False,True,False,False,False,False,...,16.0,1.0,0.062500,True,13.0,21.0,17.0,10.0,16.0,16.0


In [24]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,1.0,0.0,3.0,20.0,20.0
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,4.0,36.0,1.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,0.0,5.0,1.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,3.0,1.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,0.0,3.0,0.0,0.0,1.0,1.0


In [25]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,1.0,0.0,3.0,20.0,20.0,0.014385
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,0.0,0.0,False,4.0,36.0,1.0,0.0,3.0,3.0,0.383775
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,5.0,1.0,1.0,6.0,6.0,0.022215
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,3.0,1.0,0.0,0.0,0.0,0.005165
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,3.0,0.0,0.0,1.0,1.0,0.002870


In [26]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.49497960065961216


,id,sepostulo
0,0,0.014385
1,1,0.383775
2,2,0.022215
3,3,0.005165
4,4,0.002870


In [27]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean5.csv",index=False)